In [88]:
import numpy as np

class SOM:
    def __init__(self, nrOutputNodes, nrFetures, data, nrSamples, stepSize):
        self.weights = np.random.random((nrOutputNodes,nrFetures))
        self.data = data
        self.nrOutputNodes = nrOutputNodes
        self.nrSamples = nrSamples
        self.stepSize = stepSize

    def findMostSimularNode(self, sample):
        distances = np.zeros(self.nrOutputNodes)
        
        for node_index in range(self.nrOutputNodes):         # Measure simularity
            differences = sample - self.weights[node_index]
            distances[node_index] = np.dot(differences.T, differences)
        winnerNode = distances.argmin()
        return winnerNode

    def updateWeights(self, sample, winnerNode, neighbourhood):
        start = max(winnerNode - neighbourhood,0)
        end = min(winnerNode + neighbourhood,100)
        
        for i in range(start, end):
            deltaW = sample - self.weights[i]
            self.weights[i] = self.weights[i] + (self.stepSize*deltaW)

    def run(self, neighbourhood):
        pos = np.zeros(self.data.shape[0])
        count = 0
        
        for sample in self.data:
            winnerNode = self.findMostSimularNode(sample) # index
            pos[count] = winnerNode
            count = count + 1
            self.updateWeights(sample, winnerNode, neighbourhood)

        return pos

def readAnimals(array, path):
    arr = np.copy(array)
    
    with open(path,'r') as f:
        animal_file = f.read()
        animal = 0
        attribute = 0

    for token in animal_file:
        if token != ',' and token != '\n':
            arr[animal,attribute] = int(token)
            attribute = attribute + 1
            if attribute == 84:
                animal = animal +1
                attribute = 0
    return arr

def getResult(organizedMap):
    with open('animalnames.txt', 'r') as f:
        animals = f.readlines()

    animals = [x.strip() for x in animals]
    animals_sorted = animals[:]
    
    sorted_indices = np.argsort(organizedMap)

    for i in range(len(animals)):
        animals_sorted[i] = animals[sorted_indices[i]]
        
    return animals_sorted

def topologyOrderingAnimals():
    outputNodes = 100
    features = 84
    animals = 32

    props = np.zeros((animals,features), int)
    path = 'animals.dat'
    props = readAnimals(props, path)

    selfOrgMap = SOM(outputNodes, features, props, animals, 0.2)
    
    ephocs = 20
    startNeighbours = 50
    for epoch in range(ephocs):
        nrNeighbours = round((startNeighbours - (epoch * startNeighbours/ (ephocs-1)))/2)
        organizedMap = selfOrgMap.run(nrNeighbours)

    animals_sorted = getResult(organizedMap)
    print(animals_sorted)

topologyOrderingAnimals()

["'antelop'", "'giraffe'", "'pig'", "'camel'", "'horse'", "'lion'", "'cat'", "'bat'", "'elephant'", "'rabbit'", "'kangaroo'", "'rat'", "'ape'", "'skunk'", "'hyena'", "'bear'", "'dog'", "'walrus'", "'crocodile'", "'seaturtle'", "'frog'", "'ostrich'", "'penguin'", "'duck'", "'pelican'", "'dragonfly'", "'beetle'", "'grasshopper'", "'butterfly'", "'moskito'", "'housefly'", "'spider'"]
